In [4]:
import pandas as pd
df = pd.read_csv("../dat/fixed_model_results.csv")
df = df.fillna({'Hyperparameter_2': 99, 'Hyperparameter_3': 99})

In [5]:
# continuous independent variables for now
df_cont = df[df["Num_Categorical_Vars"] == 0]

In [41]:
display(df_cont)

,Iteration,Classes_Dep_Var,Num_Ind_Vars,Num_Categorical_Vars,Classes_Ind_Vars,Sample_Size,Model,Hyperparameter_1,Hyperparameter_2,Hyperparameter_3,Misclassification,Accuracy,Precision,Recall,F1_Score
0,1,2,3,0,2,100,LogisticRegression,logit,99.00,99,0.133333,0.866667,0.866667,0.866667,0.866667
1,1,2,3,0,2,100,LogisticRegression,probit,99.00,99,0.133333,0.866667,0.866667,0.866667,0.866667
2,1,2,3,0,2,100,DecisionTree,entropy,0.05,0.1,0.266667,0.733333,0.737557,0.733333,0.732143
3,1,2,3,0,2,100,DecisionTree,entropy,0.05,0.2,0.266667,0.733333,0.737557,0.733333,0.732143
4,1,2,3,0,2,100,DecisionTree,entropy,0.10,0.1,0.266667,0.733333,0.737557,0.733333,0.732143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10483,1,4,7,0,2,10000,ANN,2,3.00,Adam,0.484333,0.515667,0.496836,0.515667,0.492897
10484,1,4,7,0,2,10000,ANN,2,3.00,RMSprop,0.585667,0.414333,0.327838,0.414333,0.344036
10485,1,4,7,0,2,10000,ANN,2,15.00,SGD,0.475000,0.525000,0.511672,0.525000,0.509610
10486,1,4,7,0,2,10000,ANN,2,15.00,Adam,0.502000,0.498000,0.489042,0.498000,0.464417


In [33]:
groupby_list = ['Classes_Dep_Var', 'Num_Ind_Vars', 'Sample_Size', 'Model', 'Hyperparameter_1', 'Hyperparameter_2', 'Hyperparameter_3']
# get the best performing specification for each model x n_vars x sample size combination without RandomForest
df_specific_means = df_cont.groupby(groupby_list)['Misclassification'].mean()
idx = df_specific_means.groupby(['Classes_Dep_Var', 'Num_Ind_Vars', 'Sample_Size', 'Model']).idxmin()
results_cont = df_specific_means[idx]
results_cont

Classes_Dep_Var  Num_Ind_Vars  Sample_Size  Model               Hyperparameter_1  Hyperparameter_2  Hyperparameter_3
2                3             100          ANN                 1                 3.00              Adam                0.100000
                                            DecisionTree        f_test            0.05              0.1                 0.233333
                                            LogisticRegression  logit             99.00             99                  0.133333
                                            RandomForest        f_test            0.05              0.1                 0.166667
                               500          ANN                 1                 15.00             RMSprop             0.180000
                                                                                                                          ...   
4                7             1000         RandomForest        gini              0.10              0.2      

In [37]:
results_cont.groupby(['Classes_Dep_Var', 'Model']).mean()

Classes_Dep_Var  Model             
2                ANN                   0.201972
                 DecisionTree          0.268361
                 LogisticRegression    0.211472
                 RandomForest          0.241444
3                ANN                   0.343778
                 DecisionTree          0.425667
                 LogisticRegression    0.354028
                 RandomForest          0.437528
4                ANN                   0.471389
                 DecisionTree          0.541889
                 LogisticRegression    0.501528
                 RandomForest          0.575667
Name: Misclassification, dtype: float64

These are still different results from the ones in exp_FP_08

In [9]:
# Assuming df_cont is the pandas DataFrame
# Define a function to calculate the mean misclassification error with the best hyperparameters
def get_best_hyperparams_mean(data):
    grouped = data.groupby(['Classes_Dep_Var', 'Num_Ind_Vars', 'Sample_Size', 'Model'])
    result = []

    for group, frame in grouped:
        # Find the combination of hyperparameters that minimizes misclassification error
        best_hyperparams = frame.loc[frame['Misclassification'].idxmin(), ['Hyperparameter_1', 'Hyperparameter_2', 'Hyperparameter_3']]
        best_frame = frame[(frame['Hyperparameter_1'] == best_hyperparams['Hyperparameter_1']) &
                           (frame['Hyperparameter_2'] == best_hyperparams['Hyperparameter_2']) &
                           (frame['Hyperparameter_3'] == best_hyperparams['Hyperparameter_3'])]
        mean_misclassification = best_frame['Misclassification'].mean()

        result.append({
            'Classes_Dep_Var': group[0],
            'Num_Ind_Vars': group[1],
            'Sample_Size': group[2],
            'Model': group[3],
            'Mean_Misclassification': mean_misclassification
        })

    return pd.DataFrame(result)

# Apply the function to generate the summary table
result_df = get_best_hyperparams_mean(df_cont)

# Pivot table for the desired format
table = result_df.pivot_table(
    index=['Classes_Dep_Var', 'Num_Ind_Vars', 'Sample_Size'],
    columns='Model',
    values='Mean_Misclassification'
)

In [10]:
table

Model                                          ANN  DecisionTree  \
Classes_Dep_Var Num_Ind_Vars Sample_Size                           
2               3            100          0.100000      0.233333   
                             500          0.180000      0.313333   
                             1000         0.210000      0.273333   
                             10000        0.221333      0.239333   
                5            100          0.266667      0.233333   
                             500          0.253333      0.333333   
                             1000         0.206667      0.273333   
                             10000        0.208333      0.253333   
                7            100          0.200000      0.266667   
                             500          0.186667      0.293333   
                             1000         0.183333      0.253333   
                             10000        0.207333      0.254333   
3               3            100          0.333333      0.366667   
                             500          0.346667      0.406667   
                             1000         0.320000      0.406667   
                             10000        0.323667      0.371000   
                5            100          0.433333      0.500000   
                             500          0.366667      0.420000   
                             1000         0.313333      0.410000   
                             10000        0.326667      0.386667   
                7            100          0.400000      0.566667   
                             500          0.306667      0.446667   
                             1000         0.330000      0.426667   
                             10000        0.325000      0.400333   
4               3            100          0.400000      0.433333   
                             500          0.493333      0.580000   
                             1000         0.466667      0.540000   
                             10000        0.488333      0.528667   
                5            100          0.500000      0.666667   
                             500          0.473333      0.566667   
                             1000         0.460000      0.540000   
                             10000        0.469333      0.536333   
                7            100          0.500000      0.500000   
                             500          0.486667      0.493333   
                             1000         0.446667      0.556667   
                             10000        0.472333      0.561000   

Model                                     LogisticRegression  RandomForest  
Classes_Dep_Var Num_Ind_Vars Sample_Size                                    
2               3            100                    0.133333      0.166667  
                             500                    0.180000      0.246667  
                             1000                   0.220000      0.253333  
                             10000                  0.221667      0.248667  
                5            100                    0.233333      0.266667  
                             500                    0.266667      0.280000  
                             1000                   0.193333      0.250000  
                             10000                  0.210333      0.245333  
                7            100                    0.266667      0.233333  
                             500                    0.200000      0.266667  
                             1000                   0.203333      0.193333  
                             10000                  0.209000      0.246667  
3               3            100                    0.300000      0.433333  
                             500                    0.340000      0.426667  
                             1000                   0.326667      0.450000  
                             10000                  0.327667      0.438667  
                5            

In [14]:
table.groupby(['Classes_Dep_Var'])['ANN'].mean()

Classes_Dep_Var
2    0.201972
3    0.343778
4    0.471389
Name: ANN, dtype: float64